<a href="https://colab.research.google.com/github/alilotfi90/Flower_Classification/blob/main/grayscale_vs_colour_experiment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter


!pip install efficientnet_pytorch
from zipfile import ZipFile
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.optim as optim
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import torchvision.models as models




drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile


with zipfile.ZipFile('/content/drive/MyDrive/flowerimages.zip', 'r') as zip_ref:
    zip_ref.extractall('unzipped_images_flower')

In [3]:
from PIL import Image
import os

unzipped_folder = '/content/unzipped_images_flower/flowers'
output_folder = '/content/resized_images_flower'


os.makedirs(output_folder, exist_ok=True)

subdirectories = [d for d in os.listdir(unzipped_folder) if os.path.isdir(os.path.join(unzipped_folder, d))]

# seen = set()
for flower_category in subdirectories:
    input_folder = os.path.join(unzipped_folder, flower_category)
    output_category_folder = os.path.join(output_folder, flower_category)


    os.makedirs(output_category_folder, exist_ok=True)

    image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

    for image_file in image_files:
      input_image_path = os.path.join(input_folder, image_file)
      output_image_path = os.path.join(output_category_folder, image_file)

      image = Image.open(input_image_path)

      # Ensure the image is RGB
      image = image.convert("RGB")

      # Resize the image
      target_size = (224, 224)
      image = image.resize(target_size, Image.ANTIALIAS)

      image.save(output_image_path)


from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split


max_images_per_class =40
data = []

for flower_category in subdirectories:
    category_folder = os.path.join(output_folder, flower_category)
    image_files = [f for f in os.listdir(category_folder) if f.endswith('.jpg')]
    label = subdirectories.index(flower_category)  # label = folder's name

    # Limit the number of images to max_images_per_class
    image_files = image_files[:max_images_per_class]

    for image_file in image_files:
        image_path = os.path.join(category_folder, image_file)
        data.append((image_path, label))

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

<ipython-input-3-251caeb44185>:33: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize(target_size, Image.ANTIALIAS)


In [4]:


from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.grayscale_count = 0  # Adding a counter for grayscale images

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = Image.open(image_path)

        # we don't really need the following, we changed rare grayscale images to rgb when feeding to the colour_model
        # Check and convert grayscale images
        # if image.mode != 'RGB':
        #     image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)
        return image, label

transform_gray = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])

transform_color = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Data Loaders
train_loader_gray = DataLoader(CustomDataset(train_data, transform=transform_gray), batch_size=32, shuffle=True)
test_loader_gray = DataLoader(CustomDataset(test_data, transform=transform_gray), batch_size=32)
train_loader_color = DataLoader(CustomDataset(train_data, transform=transform_color), batch_size=32, shuffle=True)
test_loader_color = DataLoader(CustomDataset(test_data, transform=transform_color), batch_size=32)

class CustomEfficientNet(nn.Module):
    def __init__(self, pretrained_model_name='efficientnet-b0', num_classes=16):
        super(CustomEfficientNet, self).__init__()
        self.embedding = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        self.eff_net = EfficientNet.from_pretrained(pretrained_model_name, num_classes=num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.eff_net(x)
        return x

# Model Instances
model_gray = CustomEfficientNet().to(device)
model_color = EfficientNet.from_pretrained('efficientnet-b0', num_classes=16).to(device)

# Optimizers
optimizer_gray = optim.SGD(model_gray.parameters(), lr=0.001, momentum=0.9)
optimizer_color = optim.SGD(model_color.parameters(), lr=0.001, momentum=0.9)

# Loss function
criterion = nn.CrossEntropyLoss()


def train_test_model(model, train_loader, test_loader, optimizer, criterion, epochs_list, model_name):
    cumulative_epochs = 0
    for epochs in epochs_list:
        cumulative_epochs += epochs
        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            for i, data in enumerate(train_loader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)


                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                if i % 10 == 9:
                    print(f'[{cumulative_epochs + epoch + 1}, {i + 1}] loss: {running_loss / 10:.3f}')
                    running_loss = 0.0

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                images, labels = data[0].to(device), data[1].to(device)


                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Accuracy of {model_name} after {cumulative_epochs} epochs on test images: {accuracy:.2f}%')
    return accuracy

Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


In [5]:
epochs_list = [10, 20, 30,40]
# Assuming `accuracies_gray` is obtained similarly...
accuracies_gray = train_test_model(
    model=model_gray,
    train_loader=train_loader_gray,
    test_loader=test_loader_gray,
    optimizer=optimizer_gray,
    criterion=criterion,
    epochs_list=epochs_list,
    model_name="Grayscale Model"
)
accuracies_color = train_test_model(
    model=model_color,
    train_loader=train_loader_color,
    test_loader=test_loader_color,
    optimizer=optimizer_color,
    criterion=criterion,
    epochs_list=epochs_list,
    model_name="Color Model"
)



# Creating DataFrame and Save to Google Drive
df = pd.DataFrame({
    'Epochs': [sum(epochs_list[:i+1]) for i in range(len(epochs_list))],
    'Color_Model_Accuracy': accuracies_color,
    'Grayscale_Model_Accuracy': accuracies_gray,
})

# Save to Google Drive
# Ensure you've mounted your Google Drive if using Colab.
df.to_csv('/content/drive/MyDrive/model_accuracies.csv', index=False)

[11, 10] loss: 2.795
[12, 10] loss: 2.741
[13, 10] loss: 2.609
[14, 10] loss: 2.529
[15, 10] loss: 2.420
[16, 10] loss: 2.308
[17, 10] loss: 2.212
[18, 10] loss: 2.096
[19, 10] loss: 2.037
[20, 10] loss: 1.906
Accuracy of Grayscale Model after 10 epochs on test images: 48.44%
[31, 10] loss: 1.833
[32, 10] loss: 1.684
[33, 10] loss: 1.621
[34, 10] loss: 1.531
[35, 10] loss: 1.447
[36, 10] loss: 1.310
[37, 10] loss: 1.245
[38, 10] loss: 1.192
[39, 10] loss: 1.144
[40, 10] loss: 1.082
[41, 10] loss: 0.971
[42, 10] loss: 1.007
[43, 10] loss: 0.914
[44, 10] loss: 0.823
[45, 10] loss: 0.763
[46, 10] loss: 0.766
[47, 10] loss: 0.707
[48, 10] loss: 0.656
[49, 10] loss: 0.598
[50, 10] loss: 0.562
Accuracy of Grayscale Model after 30 epochs on test images: 77.34%
[61, 10] loss: 0.571
[62, 10] loss: 0.511
[63, 10] loss: 0.533
[64, 10] loss: 0.511
[65, 10] loss: 0.474
[66, 10] loss: 0.459
[67, 10] loss: 0.401
[68, 10] loss: 0.385
[69, 10] loss: 0.390
[70, 10] loss: 0.378
[71, 10] loss: 0.301
[72, 